In [7]:
with open("hrsw1.v36.rawByJason.vcf") as vcffile:
    vcflines = vcffile.readlines()

In [8]:
vcflines[0:5]

['##fileformat=VCFv4.0\n',
 '##Tassel=<ID=GenotypeTable,Version=5,Description="Reference allele is not known. The major allele was used as reference allele">\n',
 '##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">\n',
 '##FORMAT=<ID=AD,Number=.,Type=Integer,Description="Allelic depths for the reference and alternate alleles in the order listed">\n',
 '##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Read Depth (only filtered reads used for calling)">\n']

In [9]:
variant_repo = {} #Each variant was assigned with a unique ID
variant_repo_no_ID = {} 
ID = 0 #Assign an ID to each variant
for line in vcflines:
    if line[0] == "#":
        pass
    else:
        variant = line.split("\t")
        if variant[0] not in variant_repo:
            variant_repo[variant[0]]=list()
            variant_repo[variant[0]].append([ID,int(variant[1])])
            variant_repo_no_ID[variant[0]]=set()
            variant_repo_no_ID[variant[0]].add(int(variant[1]))
        else:
            variant_repo[variant[0]].append([ID,int(variant[1])])
            variant_repo_no_ID[variant[0]].add(int(variant[1]))
        ID=ID+1

In [10]:
len(variant_repo)
del variant_repo["UN"]
del variant_repo_no_ID["UN"]
for key in variant_repo:
    print(key)

1A
1B
1D
3B
5D
3A
5A
5B
7D
7B
7A
3D
2D
2A
2B
4D
4B
4A
6A
6B
6D


In [35]:
###Jan_17_2019 pull out some snp information for Jason, everything inside this cell is temporary
queryList=[]
with open("GT2.snps_Jan16_byJason","r") as inputf:
    for line in inputf:
        queryList.append(line.strip().split("\t"))
for variant in queryList:
    for each in variant_repo_no_ID[variant[0][1:]]:
        if(abs(each-int(variant[1]))<200):
            variant.append(each)
with open("GT2.snps_Jan16_byJason_output","w") as outputf:
    for variant in queryList:
        outputf.write("\t".join(str(v) for v in variant)+"\n")

'2x3x4'

In [ ]:
####clean lines to trim off \n
#write into file
# referenceSourcef = open("161010_Chinese_Spring_v1.0_pseudomolecules.fasta","r") 
# outputReferencef = open("reference_wheat.fasta", "w")
# readLine = referenceSourcef.readline()
# outputReferencef.write(readLine)
# readLine = referenceSourcef.readline()
# while readLine:
#     if readLine[0] == ">":
#         outputReferencef.write("\n" + readLine)
#     else:
#         outputReferencef.write(readLine.strip())
#     readLine = referenceSourcef.readline()
# referenceSourcef.close()
# outputReferencef.close()

In [2]:
with open("reference_wheat.fasta","r") as cleaned_reference:
    reference_test = {}
    currentLine = cleaned_reference.readline()
    currentChr = ""
    while currentLine:
        if currentLine[0] == ">":
            reference_test[currentLine[4:6]] = ""
            currentChr = currentLine[4:6]
            currentLine = cleaned_reference.readline()
        else:
            reference_test[currentChr] = currentLine
            currentLine = cleaned_reference.readline()
del reference_test["Un"]

In [14]:
reference_test["7B"][577278600:].find("CTCTGAGTTTCATCAGTTTCACTTCTAAACGTGATATTTCTAT")

2

In [83]:
###PstI-MseI   PstI
primer_pool = {}
for chr in variant_repo:
    for SNP in variant_repo[chr]:
        uniqueID = SNP[0]
        SNPposition = int(SNP[1])
        hundredAhead = reference_test[chr][(SNPposition-200):SNPposition]
        cuttingPos = hundredAhead.find("CTGCAG")
        if cuttingPos >= 16:
            potentialPrimer = hundredAhead[(cuttingPos-16):cuttingPos]
            #not reverse
            reversePotentialPrimer = potentialPrimer
            if reversePotentialPrimer not in primer_pool:
                primer_pool[reversePotentialPrimer] = 1
            else:
                primer_pool[reversePotentialPrimer] = primer_pool[reversePotentialPrimer] + 1 

###PstI-Mse MseI
# primer2_pool = {}
# for chr in variant_repo:
#     for SNP in variant_repo[chr]:
#         uniqueID = SNP[0]
#         SNPposition = int(SNP[1])
#         hundredAhead = reference_test[chr][SNPposition:(SNPposition+1000)]
#         cuttingPos = hundredAhead.find("TTAA")
#         if cuttingPos >= 15:
#             potentialPrimer = hundredAhead[(cuttingPos-15):cuttingPos]
#             #not reverse
#             reversePotentialPrimer = potentialPrimer
#             if reversePotentialPrimer not in primer2_pool:
#                 primer2_pool[reversePotentialPrimer] = 1
#             else:
#                 primer2_pool[reversePotentialPrimer] = primer2_pool[reversePotentialPrimer] + 1 

#The below is attempting to search qualitified reverse primer 
#The above block is searching reverse primer nearby MseI site 
primer2_pool = {}
for chr in variant_repo:
    for SNP in variant_repo[chr]:
        uniqueID = SNP[0]
        SNPposition = int(SNP[1])
        hundredAhead = reference_test[chr][SNPposition:(SNPposition+1000)]
        cuttingPos = hundredAhead.find("TTAA")
        if cuttingPos >= 15 and cuttingPos <= 900:
            searchInterval = hundredAhead[cuttingPos:(cuttingPos+200)] #define searching interval
            potentialFragment = set()
            for i in range(0,80,2):
                potentialFragment.add(searchInterval[i:(i+25)])
            for element in potentialFragment:
                reversePotentialPrimer = element
                if reversePotentialPrimer not in primer2_pool:
                    primer2_pool[reversePotentialPrimer] = 1
                else:
                    primer2_pool[reversePotentialPrimer] = primer2_pool[reversePotentialPrimer] + 1 
               
                
                
                
    

In [69]:
###MseI - PstI  MseI
primer_pool = {}
for chr in variant_repo:
    for SNP in variant_repo[chr]:
        uniqueID = SNP[0]
        SNPposition = int(SNP[1])
        hundredAhead = reference_test[chr][(SNPposition-1000):SNPposition]
        cuttingPos = hundredAhead.find("TTAA")
        if cuttingPos >= 16:
            potentialPrimer = hundredAhead[(cuttingPos-15):cuttingPos]
            #not reverse
            reversePotentialPrimer = potentialPrimer
            if reversePotentialPrimer not in primer_pool:
                primer_pool[reversePotentialPrimer] = 1
            else:
                primer_pool[reversePotentialPrimer] = primer_pool[reversePotentialPrimer] + 1 

###MseI - PstI PstI
primer2_pool = {}
for chr in variant_repo:
    for SNP in variant_repo[chr]:
        uniqueID = SNP[0]
        SNPposition = int(SNP[1])
        hundredAhead = reference_test[chr][SNPposition:(SNPposition+200)]
        cuttingPos = hundredAhead.find("CTGCAG")
        if cuttingPos >= 16:
            potentialPrimer = hundredAhead[(cuttingPos-16):cuttingPos]
            #not reverse
            reversePotentialPrimer = potentialPrimer
            if reversePotentialPrimer not in primer2_pool:
                primer2_pool[reversePotentialPrimer] = 1
            else:
                primer2_pool[reversePotentialPrimer] = primer2_pool[reversePotentialPrimer] + 1 

In [ ]:
# import csv
# with open('dict.csv', 'w',newline = "") as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in primer_pool.items():
#        writer.writerow([key, value])

In [ ]:
# import csv
# with open('dict_primer2.csv', 'w',newline = "") as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in primer2_pool.items():
#        writer.writerow([key, value])

In [148]:
from multiprocessing import Pool

#For a given string(like a chromosome), find all hits
#return a generator
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

#List all hits in the chromosome, and catach all possible combinations with expected size
#the function is out-of-date because of overestimating number of amplicons
def catch_fragment(chromosome,hits_list1, hits_list2):
    validFragmentNum = 0
    validSNP = 0
    for hit1 in hits_list1:
        for hit2 in hits_list2:
            if((int(hit2) - int(hit1)) >100) and ((int(hit2) - int(hit1)) <250):
                validFragmentNum = validFragmentNum + 1
                validSNP = validSNP + SNP_inside(chromosome,hit1,hit2)
    return [validFragmentNum,validSNP]

def catch_fragment_large(chromosome,hits_list1, hits_list2):
    validFragmentNum = 0
    validSNP = 0
    for hit1 in hits_list1:
        for hit2 in hits_list2:
            if((int(hit2) - int(hit1)) >249) and ((int(hit2) - int(hit1)) <1000):
                validFragmentNum = validFragmentNum + 1
                validSNP = validSNP + SNP_inside(chromosome,hit1,hit2)
    return [validFragmentNum,validSNP]

#count how many SNPs inside a given fragement inside the chromosome
def SNP_inside(current_chromosome, current_hit1, current_hit2):
    return len(variant_repo_no_ID[current_chromosome].intersection(range(current_hit1,current_hit2)))

#return all SNPs inside a given fragement inside the chromosome 
def SNP_inside_with_name(current_chromosome, current_hit1, current_hit2):
    return [variant_repo_no_ID[current_chromosome].intersection(range(current_hit1,current_hit2))]

#PCR simulator
#for a pair of given priemrs, report all hits' position
#catch all fragments with 500 - 1000 bps.

allChr = list(reference_test.keys())
# def pcr_simulator(chr):
#     validFragment = 0
#     primer_one_hits = list(find_all(reference_test[chr],primer_one))
#     primer_two_hits = list(find_all(reference_test[chr],primer_two))
#     validFragment = validFragment + catch_fragment(primer_one_hits,primer_two_hits)
#     del primer_one_hits
#     del primer_two_hits
#     return validFragment

###Brute force is not too bad
###Marker PstI and MseI
#create a new method for brute force
def pcr_simulator_para(primerPair):
    primer_one = primerPair[0]
    primer_two = primerPair[1]
    validFragment = 0
    validSNP = 0
    validFragment_large = 0
    validSNP_large = 0
    
    for chr in allChr:
        primer_one_hits = list(find_all(reference_test[chr],primer_one))
        primer_two_hits = list(find_all(reference_test[chr],primer_two))
        frag_and_SNP = catch_fragment(chr,primer_one_hits,primer_two_hits)
        validFragment = validFragment + frag_and_SNP[0]
        validSNP = validSNP + frag_and_SNP[1]
        
        frag_and_SNP_large = catch_fragment_large(chr,primer_one_hits,primer_two_hits)
        validFragment_large = validFragment_large + frag_and_SNP_large[0]
        validSNP_large = validSNP_large + frag_and_SNP_large[1]
        
    return [primerPair[0],primer_pool[primerPair[0]],primerPair[1],primer2_pool[primerPair[1]],validFragment,validSNP,validFragment_large,validSNP_large]

def pcr_simulator_para_for_site(primerPair):
    primer_one = primerPair[0]
    primer_two = primerPair[1]
    validFragment = 0
    validSNP = 0
    validFragment_large = 0
    validSNP_large = 0
    
    for chr in allChr:
        primer_one_hits = list(find_all(reference_test[chr],primer_one))
        primer_two_hits = list(find_all(reference_test[chr],primer_two))
        frag_and_SNP = catch_fragment(chr,primer_one_hits,primer_two_hits)
        validFragment = validFragment + frag_and_SNP[0]
        validSNP = validSNP + frag_and_SNP[1]
        
        frag_and_SNP_large = catch_fragment_large(chr,primer_one_hits,primer_two_hits)
        validFragment_large = validFragment_large + frag_and_SNP_large[0]
        validSNP_large = validSNP_large + frag_and_SNP_large[1]
        
    return [primerPair[0],primerPair[1],validFragment,validSNP,validFragment_large,validSNP_large]

def pcr_simulator_para_two(primerPair):
    primer_result = pcr_simulator_para(primerPair)
    #Add enzyme site  CTGCAG or TTAA
    primerPairSite = [primerPair[0]+ enzyme1, primerPair[1] + enzyme2]
    primer_with_site_result = pcr_simulator_para_for_site(primerPairSite)
    return [primer_result,primer_with_site_result]



#
#Return a list with all amplicons, which is a list contain all info about it.
def one_primer_detail(primerPair):
    ampliconHolder = []
    for chr in allChr:
        primer1Hits = list(find_all(reference_test[chr],primerPair[0]))
        primer2Hits = list(find_all(reference_test[chr],primerPair[1]))
        iterPrimer1Hits = iter(primer1Hits)
        iterPrimer2Hits = iter(primer2Hits)
        primer1First = next(iterPrimer1Hits,False)
        primer1Second = next(iterPrimer1Hits,False)
        primer2First = next(iterPrimer2Hits,False)
        primer2Second = next(iterPrimer2Hits,False)
        while(primer1Second and primer2Second): ##check iterator hasNext()
            if primer2First < primer1First:   
                ## ______P1_______P1____
                ##___P2_______________
                primer2First = primer2Second
                primer2Second = next(iterPrimer2Hits,False)
            elif primer1Second < primer2First:
                ## ______P1_______P1____
                ##____________________P2______
                primer1First = primer1Second
                primer1Second = next(iterPrimer1Hits,False)
            else:
                ##_______P1________P1___
                ##__________P2__________
                if (primer2First - primer1First) < 5000: ##too long fragment may not be real and memory-consuming
                    anAmplicon=[primer1First,primer2First,chr,reference_test[chr][primer1First:primer2First], primer2First - primer1First,SNP_inside(chr,primer1First,primer2First),SNP_inside_with_name(chr,primer1First,primer2First)]
                    if (primer2First - primer1First) > 120: #extract 120 or less bp part of an amplicon
                        anAmplicon.append(reference_test[chr][primer1First:(primer1First+120)])
                    else:
                        anAmplicon.append(reference_test[chr][primer1First:primer2First])
                    ampliconHolder.append(anAmplicon)
                primer1First = primer1Second
                primer1Second = next(iterPrimer1Hits,False)
                primer2First = primer2Second
                primer2Second = next(iterPrimer2Hits,False)
    return ampliconHolder


#This function counts distributions of amplicons on each chromosome
#This function obtained amount of information about one primer pair.
def one_primer_amplicon_distribution(primerPair):
    ampliconInPCR = one_primer_detail(primerPair)
    ampliconByChr = {}
    for amplicon in ampliconInPCR:
        ampliconSample = {"1A":0,"1B":0,"1D":0,
                         "2A":0,"2B":0,"2D":0,
                         "3A":0,"3B":0,"3D":0,
                         "4A":0,"4B":0,"4D":0,
                         "5A":0,"5B":0,"5D":0,
                         "6A":0,"6B":0,"6D":0,
                         "7A":0,"7B":0,"7D":0}
        if amplicon[3] not in ampliconByChr:
            ampliconByChr[amplicon[3]] = ampliconSample
            ampliconByChr[amplicon[3]][amplicon[2]] = ampliconByChr[amplicon[3]][amplicon[2]] + 1
        else:
            ampliconByChr[amplicon[3]][amplicon[2]] = ampliconByChr[amplicon[3]][amplicon[2]] + 1
    return ampliconByChr

def one_primer_amplicon_first120_distribution(primerPair):
    ampliconInPCR = one_primer_detail(primerPair)
    ampliconFirst120ByChr = {}
    for amplicon in ampliconInPCR:
        ampliconSample = {"1A":0,"1B":0,"1D":0,
                         "2A":0,"2B":0,"2D":0,
                         "3A":0,"3B":0,"3D":0,
                         "4A":0,"4B":0,"4D":0,
                         "5A":0,"5B":0,"5D":0,
                         "6A":0,"6B":0,"6D":0,
                         "7A":0,"7B":0,"7D":0}
        if amplicon[-1] not in ampliconFirst120ByChr:
            ampliconFirst120ByChr[amplicon[-1]] = ampliconSample
            ampliconFirst120ByChr[amplicon[-1]][amplicon[2]] = ampliconFirst120ByChr[amplicon[-1]][amplicon[2]] + 1
        else:
            ampliconFirst120ByChr[amplicon[-1]][amplicon[2]] = ampliconFirst120ByChr[amplicon[-1]][amplicon[2]] + 1
    return ampliconFirst120ByChr




#The ampliconByChr is stored in a dictionary. The function below is to insert the dictionary to each line.
def one_primer_merge_detail_distribution(primerPair):
    ampliconInPCR = one_primer_detail(primerPair)
    ampliconByChr = one_primer_amplicon_distribution(primerPair)
    ampliconFirst120ByChr = one_primer_amplicon_first120_distribution(primerPair)
    for amplicon in ampliconInPCR:
        amplicon.insert(3,ampliconByChr[amplicon[3]])
        amplicon.append(ampliconFirst120ByChr[amplicon[-1]])
    return ampliconInPCR

from collections import Counter
def compute_mode(numbers):
    modes = []
    counts = Counter(numbers)
    maxCount = max(counts.values())
    for num,count in counts.items():
        if count==maxCount:
            modes.append(num)
    return modes


def one_primer_summary(primerPair):
    onePrimerPair = one_primer_merge_detail_distribution(primerPair)
    aGenomeHits,bGenomeHits,dGenomeHits = 0,0,0
    totalSNP = 0
    uniqueAmplicon = 0
    uniqueAmpliconFirst120 = 0
    totalAmplicon = len(onePrimerPair)
    if totalAmplicon > 1:
        sizeMode = 0
        sizeList = []
        for eachAmpliconInfo in onePrimerPair:
            aGenomeHits = aGenomeHits + sum([eachAmpliconInfo[3]["1A"],eachAmpliconInfo[3]["2A"],eachAmpliconInfo[3]["3A"],eachAmpliconInfo[3]["4A"],eachAmpliconInfo[3]["5A"],eachAmpliconInfo[3]["6A"],eachAmpliconInfo[3]["7A"]])
            bGenomeHits = bGenomeHits + sum([eachAmpliconInfo[3]["1B"],eachAmpliconInfo[3]["2B"],eachAmpliconInfo[3]["3B"],eachAmpliconInfo[3]["4B"],eachAmpliconInfo[3]["5B"],eachAmpliconInfo[3]["6B"],eachAmpliconInfo[3]["7B"]])
            dGenomeHits = dGenomeHits + sum([eachAmpliconInfo[3]["1D"],eachAmpliconInfo[3]["2D"],eachAmpliconInfo[3]["3D"],eachAmpliconInfo[3]["4D"],eachAmpliconInfo[3]["5D"],eachAmpliconInfo[3]["6D"],eachAmpliconInfo[3]["7D"]])
            totalSNP = totalSNP + eachAmpliconInfo[6]
            sizeList.append(eachAmpliconInfo[5])
            allHits = sum(eachAmpliconInfo[3].values())
            if allHits == 1: uniqueAmplicon = uniqueAmplicon + 1
            if sum(eachAmpliconInfo[-1].values()) == 1: uniqueAmpliconFirst120 = uniqueAmpliconFirst120 + 1
        uniqueRatio = uniqueAmplicon/totalAmplicon
        sizeMode = compute_mode(sizeList)
        return [primerPair[0],primerPair[1],totalAmplicon,aGenomeHits,bGenomeHits,dGenomeHits,totalSNP,uniqueAmplicon,uniqueRatio,uniqueAmpliconFirst120,uniqueAmpliconFirst120/totalAmplicon,sizeMode]
    else:
        return [primerPair[0],primerPair[1],0]
#     ['TAGGCGAGTACTTGG',
#  'ACTCGGTAGGATTTTT',
#  2356,
#  29535,
#  45643,
#  150,
#  1476174135,
#  1532,
#  0.6502546689303905,
#  0]


In [97]:
allPoentialPrimer = [] 
for primer1, occur1 in primer_pool.items():
    if occur1 >600:
        for primer2, occur2 in primer2_pool.items():
            if occur2 >1000 and primer2 != "":
                allPoentialPrimer.append([primer1,primer2])

In [98]:
len(allPoentialPrimer)

1400

In [116]:
#define enzyme site in pcr_simulator_para_two first
enzyme1 = "TTAA"
enzyme2 = "CTGCAG"
with Pool(12) as p:
    potentialPrimerResult = p.map(pcr_simulator_para_two,allPoentialPrimer)


In [118]:
import csv
with open('potentialPrimerMse_Pst_16_15_everything.csv', 'w',newline = "") as csv_file:
    writer = csv.writer(csv_file)
    for value in potentialPrimerResult:
       writer.writerow(value)


In [149]:
finalPrimerpPair1 = one_primer_merge_detail_distribution(["TAGGCGAGTACTTGG","ACTCGGTAGGATTTTT"])

In [152]:
finalPrimerpPair1[-10]

[621985061,
 621985187,
 '5B',
 {'1A': 0,
  '1B': 0,
  '1D': 0,
  '2A': 0,
  '2B': 0,
  '2D': 0,
  '3A': 0,
  '3B': 0,
  '3D': 0,
  '4A': 0,
  '4B': 0,
  '4D': 0,
  '5A': 0,
  '5B': 1,
  '5D': 0,
  '6A': 0,
  '6B': 0,
  '6D': 0,
  '7A': 0,
  '7B': 0,
  '7D': 0},
 'TAGGCGAGTACTTGGACTGCAGCTAAGCCTCCGAGTGGGAGGCTGGCTCACCACTCGGTAGGATTTTAAACACTTAGGAGAGTACTTGGACTGTAGCTAAGCCTCCGAGTGGGAGGATTGCTCTCC',
 126,
 0,
 [set()],
 'TAGGCGAGTACTTGGACTGCAGCTAAGCCTCCGAGTGGGAGGCTGGCTCACCACTCGGTAGGATTTTAAACACTTAGGAGAGTACTTGGACTGTAGCTAAGCCTCCGAGTGGGAGGATTG',
 {'1A': 0,
  '1B': 0,
  '1D': 0,
  '2A': 0,
  '2B': 0,
  '2D': 0,
  '3A': 0,
  '3B': 0,
  '3D': 0,
  '4A': 0,
  '4B': 0,
  '4D': 0,
  '5A': 0,
  '5B': 1,
  '5D': 0,
  '6A': 0,
  '6B': 0,
  '6D': 0,
  '7A': 0,
  '7B': 0,
  '7D': 0}]

In [151]:
aGenomeHits,bGenomeHits,dGenomeHits = 0,0,0
for eachAmpliconInfo in finalPrimerpPair1:
    print(eachAmpliconInfo[3].values())
        #aGenomeHits = aGenomeHits + sum([eachAmpliconInfo[3]["1A"],eachAmpliconInfo[3]["2A"],eachAmpliconInfo[3]["3A"],eachAmpliconInfo[3]["4A"],eachAmpliconInfo[3]["5A"],eachAmpliconInfo[3]["6A"],eachAmpliconInfo[3]["7A"]])
        #bGenomeHits = bGenomeHits + sum([eachAmpliconInfo[3]["1B"],eachAmpliconInfo[3]["2B"],eachAmpliconInfo[3]["3B"],eachAmpliconInfo[3]["4B"],eachAmpliconInfo[3]["5B"],eachAmpliconInfo[3]["6B"],eachAmpliconInfo[3]["7B"]])
        #dGenomeHits = dGenomeHits + sum([eachAmpliconInfo[3]["1D"],eachAmpliconInfo[3]["2D"],eachAmpliconInfo[3]["3D"],eachAmpliconInfo[3]["4D"],eachAmpliconInfo[3]["5D"],eachAmpliconInfo[3]["6D"],eachAmpliconInfo[3]["7D"]])
            

dict_values([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
dict_values([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
dict_values([16, 0, 0, 4, 20, 12, 10, 23, 18, 16, 0, 7, 15, 0, 11, 11, 0, 28, 0, 0, 18])
dict_values([0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
dict_values([16, 0, 0, 4, 20, 12, 10, 23, 18, 16, 0, 7, 15, 0, 11, 11, 0, 28, 0, 0, 18])
dict_values([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
dict_values([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
dict_values([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
dict_values([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
dict_values([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
dict_values([11, 0, 0, 10, 7, 2, 8, 9, 7, 7, 0, 4, 3, 0, 2, 7, 0, 22, 0, 0, 3])
dict_values([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
dict_values([2, 0, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 

In [91]:
len([x for x in finalPrimerpPair1 if((x[3] < 300) and (x[3]>50)) and x[4] != 0])

202

In [52]:
testSets = []
with open("Adj_primersets.csv") as testSetFile:
    testSetsAllLine = testSetFile.readlines()
    for line in testSetsAllLine:
        testSets.append(line.strip().upper().split(","))

In [53]:
testSets[17]

['18', 'GCGAGTACTGGACTGCAG', 'ACTCGGTAGGATTTTTTAA']

In [58]:
testSetInfo = []
for primerPair in testSets:
    onePrimerInfo = one_primer_merge_detail_distribution(primerPair[1:])
    testSetInfo.append([x for x in onePrimerInfo if((x[5] < 5000) and (x[5]>25))])

In [59]:
len(testSetInfo)

22

In [91]:
import csv
for i in range(0,22):
    fileName = "final_each_primer_distribution_set_" + str(i) + ".csv"
    with open(fileName, 'w',newline = "") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["start","end","chr","length","# of SNP","pos of SNP, (set() means no SNP) "])
        for value in testSetInfo[i]:
            onerow = [value[0],value[1],value[2]]
            onerow.append(value[4])
            onerow.append(value[5])
            onerow.append(value[6])
            onerow.append(value[7])
            chrSeq = []
            for key in value[3]:
                onerow.append(value[3][key])
                onerow.append(sum([value[3]["1A"],value[3]["2A"],value[3]["3A"],value[3]["4A"],value[3]["5A"],value[3]["6A"],value[3]["7A"]]))
                onerow.append(sum([value[3]["1B"],value[3]["2B"],value[3]["3B"],value[3]["4B"],value[3]["5B"],value[3]["6B"],value[3]["7B"]]))
                onerow.append(sum([value[3]["1D"],value[3]["2D"],value[3]["3D"],value[3]["4D"],value[3]["5D"],value[3]["6D"],value[3]["7D"]]))
                onerow.append(sum(value[3].values()))
                chrSeq.append(key)
            writer.writerow(onerow)
        chrSeq.append("A subgenome")
        chrSeq.append("B subgenome")
        chrSeq.append("D subgenome")
        chrSeq.append("sum of occurrances")
        writer.writerow(chrSeq)

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [49]:
test = one_primer_merge_detail_distribution(["TAGGCGAGTACTTGG","ACTCGGTAGGATTTTT"])

In [50]:
len(test)

2356

In [51]:
test[0:10]

[[2257464,
  2257589,
  '7B',
  {'1A': 0,
   '1B': 0,
   '1D': 0,
   '2A': 0,
   '2B': 0,
   '2D': 0,
   '3A': 0,
   '3B': 0,
   '3D': 0,
   '4A': 0,
   '4B': 0,
   '4D': 0,
   '5A': 0,
   '5B': 0,
   '5D': 0,
   '6A': 0,
   '6B': 0,
   '6D': 0,
   '7A': 0,
   '7B': 1,
   '7D': 0},
  'TAGGCGAGTACTTGGACTGCAGCTAAGCCTCCGAGTGGAAGGCTGGCTCACCACTTGGTAGGATTTTGTTTAACTTAGGCGAAACGGATTCGCAGCTAAGCCTCCGAGTGGAAGGCTGGCTTACC',
  125,
  0,
  [set()]],
 [6247817,
  6247869,
  '7B',
  {'1A': 0,
   '1B': 0,
   '1D': 0,
   '2A': 0,
   '2B': 0,
   '2D': 0,
   '3A': 0,
   '3B': 0,
   '3D': 0,
   '4A': 0,
   '4B': 0,
   '4D': 0,
   '5A': 0,
   '5B': 0,
   '5D': 0,
   '6A': 0,
   '6B': 0,
   '6D': 0,
   '7A': 0,
   '7B': 1,
   '7D': 0},
  'TAGGCGAGTACTTGGATTGCAACTAAGCCCCCGAGTGGGAGGGTTGCCCTCC',
  52,
  0,
  [set()]],
 [7762076,
  7762128,
  '7B',
  {'1A': 15,
   '1B': 18,
   '1D': 0,
   '2A': 4,
   '2B': 16,
   '2D': 0,
   '3A': 20,
   '3B': 28,
   '3D': 0,
   '4A': 7,
   '4B': 11,
   '4D': 0,
   '5A': 10,
   '5

In [153]:
one_primer_summary(["TAGGCGAGTACTTGG","ACTCGGTAGGATTTTT"])

['TAGGCGAGTACTTGG',
 'ACTCGGTAGGATTTTT',
 1596,
 29435,
 44985,
 148,
 498,
 772,
 0.48370927318295737,
 717,
 0.4492481203007519,
 [52]]

In [154]:
with Pool(12) as p:
    potentialPrimerResult = p.map(one_primer_summary,allPoentialPrimer)

In [114]:
potentialPrimerResult

[['TTAGGCGAGTACTAGA', 'GATCGCGAGGGTGCGAGATTGCTGA', 0],
 ['TTAGGCGAGTACTAGA', 'ACACATAGTGAATACATGAACTCCT', 0],
 ['TTAGGCGAGTACTAGA', 'CACTCGGCATTCCGAGGCAAATTTA', 0],
 ['TTAGGCGAGTACTAGA',
  'AGGCGAGTACTGGACTGCAGCTAAG',
  758,
  444,
  234,
  348,
  1477,
  683,
  0.9010554089709762,
  [75]],
 ['TTAGGCGAGTACTAGA',
  'CCCATCTGCCCATAACCGCGGGCAC',
  4,
  4,
  0,
  0,
  15,
  4,
  1.0,
  [4793, 1059, 1628, 911]],
 ['TTAGGCGAGTACTAGA',
  'AAGGCCTCCTTTTAATAGAGAACCG',
  2,
  1,
  1,
  0,
  2,
  2,
  1.0,
  [3602, 1559]],
 ['TTAGGCGAGTACTAGA', 'GATTGCTGAGTCCCTGTGGCTCACA', 0],
 ['TTAGGCGAGTACTAGA',
  'CATTAACAAATTCCCATCTGCCCAT',
  3,
  3,
  0,
  0,
  16,
  3,
  1.0,
  [1616, 2514, 3260]],
 ['TTAGGCGAGTACTAGA', 'ATAGTCTTGATCGCGAGGGTGCGAG', 0],
 ['TTAGGCGAGTACTAGA', 'AAGTTTTTTACCCCTCCTGTGGAGG', 0],
 ['TTAGGCGAGTACTAGA',
  'TCTGCCCATAACCGCGGGCACGGCT',
  3,
  3,
  0,
  0,
  14,
  3,
  1.0,
  [1632, 4797, 1063]],
 ['TTAGGCGAGTACTAGA',
  'ACCGGACATTAACAAATTCCCATCT',
  5,
  3,
  2,
  0,
  28,
  5,
  1.0

In [155]:
import csv
fileName = "potentialPrimerResultAug9.csv"
with open(fileName, 'w',newline = "") as csv_file:
    writer = csv.writer(csv_file)
    for element in potentialPrimerResult:
        
        writer.writerow(element)